In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from statistics import mode
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_houses_dirty = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_houses_dirty = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

train_houses_1 = train_houses_dirty.copy()
#train_houses_1 = train_houses_dirty.sample(n=30, random_state=2)
test_houses_1 = test_houses_dirty.copy()
#test_houses_1 = test_houses_dirty.sample(n=30, random_state=2)

train_houses_1 = train_houses_1.drop('FireplaceQu', axis=1)
test_houses_1 = test_houses_1.drop('FireplaceQu', axis=1)

#--------------------------------------------------------------------------------------------------------------------------------------

for i in train_houses_dirty.columns:
    if train_houses_dirty[i].isna().sum() >= (0.8*train_houses_dirty.shape[0]):
        #print(i)
        train_houses_1 = train_houses_1.drop(i, axis=1)
#print('\n')
for i in test_houses_dirty.columns:
    if test_houses_dirty[i].isna().sum() >= (0.8*test_houses_dirty.shape[0]):
        #print(i)
        test_houses_1 = test_houses_1.drop(i, axis=1)

#--------------------------------------------------------------------------------------------------------------------------------------
train_houses_2 = train_houses_1.copy()
test_houses_2 = test_houses_1.copy()

for i in train_houses_2.columns:
    if train_houses_2[i].dtype == 'float64':  
        train_houses_2[i] = train_houses_2[i].fillna(train_houses_2[i].mean())
        #print('float',i)
    elif train_houses_2[i].dtype == 'int64':
        train_houses_2[i] = train_houses_2[i].fillna(train_houses_2[i].median())
        #print('int',i)
    else:
        train_houses_2[i] = train_houses_2[i].fillna(train_houses_2[i].mode()[0])
        
for i in test_houses_2.columns:
    if test_houses_2[i].dtype == 'float64':  
        test_houses_2[i] = test_houses_2[i].fillna(test_houses_2[i].mean())
        #print('float',i)
    elif test_houses_2[i].dtype == 'int64':
        test_houses_2[i] = test_houses_2[i].fillna(test_houses_2[i].median())
        #print('int',i)
    else:
        test_houses_2[i] = test_houses_2[i].fillna(test_houses_2[i].mode()[0])
        
#--------------------------------------------------------------------------------------------------------------------------------------

X_houseprice_tr = train_houses_2.drop('SalePrice', axis=1)
y_houseprice_tr = train_houses_2['SalePrice']

X_houseprice_tr_dummies = pd.get_dummies(X_houseprice_tr)
X_topredict_dummies = pd.get_dummies(test_houses_2)

#--------------------------------------------------------------------------------------------------------------------------------------

# Get missing columns in the training test
missing_cols = set( X_houseprice_tr_dummies.columns ) - set( X_topredict_dummies.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_topredict_dummies[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_topredict_dummies = X_topredict_dummies[X_houseprice_tr_dummies.columns]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

X_tr, X_ts, y_tr, y_ts = train_test_split(X_houseprice_tr_dummies, y_houseprice_tr, test_size=0.2, random_state=1307)

lasso_linear = Lasso(alpha=35)
lasso_linear.fit(X_tr, y_tr)
y_pred = lasso_linear.predict(X_ts)

print('lasso r2 score = {}'.format(round(lasso_linear.score(X_ts, y_ts)*100,2)),'%')
print('precision with training data =',round(lasso_linear.score(X_tr, y_tr)*100,2),'%')

In [ ]:
results_list = lasso_linear.predict(X_topredict_dummies)
results = pd.DataFrame(results_list)

test_houses_dirty['SalePrice']= results

to_export = test_houses_dirty[['Id', 'SalePrice']]

to_export.to_csv('predictions_HousePricePredict_NiengLee.csv', index=False)

In [ ]:
from sklearn.metrics import r2_score

sample = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
print(r2_score(sample['SalePrice'], results))

sm = list(sample['SalePrice'])
re = list(results_list)
sub=[]
for i in range(0,len(results)):
    sub.append(sm[i] - re[i])
    
    
print(r2_score(sm, re))

In [ ]:
#train_houses_dirty.info()
#print('\n')
#train_houses_1.info()

#print(train_houses_2.shape)
#print(test_houses_2.shape)

#print(train_houses_2.isnull().sum().sum())
#print(test_houses_2.isnull().sum().sum())

#print(X_houseprice_tr_dummies.info())
#print(X_topredict_dummies.info())

#print(train_houses_dirty.columns)